In [ ]:
# A décommenter à la première exécution si jupyter se plaint de ne pas trouver oracledb
# !pip install --upgrade oracledb

In [60]:
# Compléter ici les imports dont vous avez besoin, ne pas modifier ceux déjà présents
import getpass
from os import getenv
import pandas as pd
import oracledb
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
# Nécessaire pour éviter les problèmes de session
class Connexion(object):
    def __init__(self, login, password):
        self.conn = oracledb.connect(
            user=login,
            password=password,
            host="oracle.iut-orsay.fr",
            port=1521,
            sid="etudom",
        )
        self.conn.autocommit = False

    def __enter__(self):
        self.conn.autocommit = False
        return self.conn

    def __exit__(self, *args):
        self.conn.close()

In [62]:
# La fonction ci-dessous est à utiliser pour exécuter une requête et stocker les résultats dans un dataframe Pandas sans afficher d’alerte.
# Vous pouvez vous en inspirer pour créer vos propres fonctions.
def requete_vers_dataframe(connexion_data, requete, valeurs = None):
    with Connexion(login=connexion_data['login'], password=connexion_data['password']) as connexion:
        warnings.simplefilter(action='ignore', category=UserWarning)
        if valeurs is not None:
            df = pd.read_sql(requete, connexion, params=valeurs)
        else:
            df = pd.read_sql(requete, connexion)
        warnings.simplefilter("always") 
        return df

In [63]:
# Saisir ci-dessous l'édition des JO qui vous a été attribuée. Cela correspond au LibelleHote dans la table HOTE de la base de données
# Par exemple EDITION_JO = "2020 Summer Olympics"
EDITION_JO = "1956 Summer Olympics"
# Saisir ci-dessous le login court de la base utilisée pour votre carnet
SCHEMA = "MSOBI"

# TABLEAU DE BORD DES JO

## Partie consultation des données

In [64]:
# On demande à l'utilisateur son login et mot de passe pour pouvoir accéder à la base de données
if getenv("DB_LOGIN") is None:
    login = input("Login : ")
else:
    login = getenv("DB_LOGIN")
if getenv("DB_PASS") is None:
    password = getpass.getpass("Mot de passe : ")
else:
    password = getenv("DB_PASS")
conn = {'login': login, 'password': password}

Login :  msobi
Mot de passe :  ········


In [65]:
# On vérifie que l'utilisateur est bien connecté à la base de données, que le schéma est bon, et qu'on trouve la bonne édition des JO
data = requete_vers_dataframe(conn, f"SELECT * FROM {SCHEMA}.HOTE WHERE LibelleHote LIKE (:libelle)",{"libelle":EDITION_JO})
id_hote = int(data.IDHOTE.iloc[0])
print(f"Identifiant de l’hôte : {id_hote}")

Identifiant de l’hôte : 14


### Statistiques de base : participation

#### Nombre de nations participant à ces JO

In [66]:
# Calculer avec une requête et afficher le nombre de nations participant à cette édition
# code fait par Joris 

data_nombre_nations = requete_vers_dataframe(conn, f"""
                                            SELECT COUNT(DISTINCT codenoc) AS "NombreDeNation"
                                            FROM {SCHEMA}.NOC N
                                            INNER JOIN {SCHEMA}.participation_individuelle PI ON N.codenoc = PI.noc
                                            INNER JOIN {SCHEMA}.EQUIPE E ON E.noc = N.codenoc
                                            INNER JOIN {SCHEMA}.participation_equipe PE ON e.idequipe = PE.idequipe
                                            inner join {SCHEMA}.evenement on evenement.idEvenement = pe.idEvenement
                                            where idHote = 14
                                            """)
data_nombre_nations

,NombreDeNation
0,55


#### Nombre d'athlètes participant à ces JO, répartition par genre et /ou par pays

In [67]:
# Calculer avec une requête et afficher le nombre d’athlètes participant et la répartition par genre et/ou par pays
# code fait par Joris 

data_nombre_athlete_par_genre_pays = requete_vers_dataframe(conn, f"""
                                            SELECT 
                                                (SELECT COUNT(DISTINCT A.IdAthlete) AS "Nombre"
                                                FROM {SCHEMA}.ATHLETE A
                                                INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI ON A.IdAthlete = PI.IdAthlete
                                                INNER JOIN {SCHEMA}.EVENEMENT E ON PI.IdEvent = E.IdEvenement
                                                INNER JOIN {SCHEMA}.HOTE H ON E.IdHote = H.IdHote
                                                WHERE H.AnneeHote = 2020) AS NombreTotalAthletes,
                                                Genre, 
                                                COUNT(DISTINCT A.IdAthlete) AS NombreAthletesParGenre,
                                                NOC,
                                                COUNT(DISTINCT PI.IdAthlete) AS NombreAthletesParPays
                                            FROM ATHLETE A
                                            INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI ON A.IdAthlete = PI.IdAthlete
                                            INNER JOIN {SCHEMA}.EVENEMENT E ON PI.IdEvent = E.IdEvenement
                                            INNER JOIN {SCHEMA}.HOTE H ON E.IdHote = H.IdHote
                                            WHERE H.AnneeHote = 2020
                                            GROUP BY Genre, NOC
                                            """)
data_nombre_athlete_par_genre_pays

,NOMBRETOTALATHLETES,GENRE,NOMBREATHLETESPARGENRE,NOC,NOMBREATHLETESPARPAYS
0,6889,Male,4,AFG,4
1,6889,Male,6,ALB,6
2,6889,Male,26,ALG,26
3,6889,Male,1,AND,1
4,6889,Male,2,ANG,2
...,...,...,...,...,...
403,6889,Female,8,VIE,8
404,6889,Female,2,VIN,2
405,6889,Female,2,YEM,2
406,6889,Female,2,ZAM,2


### Les médailles décernées lors de ces JO

#### Médailles individuelles (épreuves individuelles)

##### Tableau du TOP 10 individuel

In [74]:
# Récupérer les données, les transformer si nécessaire, puis afficher sous forme de tableau et/ou d'un graphique
# code fait par Joris 

data = requete_vers_dataframe(conn,f"""
                                            SELECT 
                                                (SELECT COUNT(DISTINCT A.IdAthlete) AS "Nombre"
                                                FROM {SCHEMA}.ATHLETE A
                                                INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI ON A.IdAthlete = PI.IdAthlete
                                                INNER JOIN {SCHEMA}.EVENEMENT E ON PI.IdEvent = E.IdEvenement
                                                INNER JOIN {SCHEMA}.HOTE H ON E.IdHote = H.IdHote
                                                WHERE H.AnneeHote = 2020) AS NombreTotalAthletes,
                                                Genre, 
                                                COUNT(DISTINCT A.IdAthlete) AS NombreAthletesParGenre,
                                                NOC,
                                                COUNT(DISTINCT PI.IdAthlete) AS NombreAthletesParPays
                                            FROM ATHLETE A
                                            INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI ON A.IdAthlete = PI.IdAthlete
                                            INNER JOIN {SCHEMA}.EVENEMENT E ON PI.IdEvent = E.IdEvenement
                                            INNER JOIN {SCHEMA}.HOTE H ON E.IdHote = H.IdHote
                                            WHERE H.AnneeHote = 2020
                                            GROUP BY Genre, NOC
                                            """)
# Afficher les 10 premières lignes pour vérifier les données
data.iloc[0:10]
print(data.head(10))

# Vérifier que les colonnes nécessaires existent avant de créer les graphiques
if 'Genre' in data.columns and 'NOC' in data.columns:
    plt.figure(figsize=(14, 7))

    # Graphique du nombre d'athlètes par genre
    plt.subplot(1, 2, 1)
    sns.countplot(data=data, x='Genre')
    plt.title('Nombre d\'athlètes par genre en 2020')

    # Graphique du nombre d'athlètes par pays
    plt.subplot(1, 2, 2)
    sns.countplot(data=data, x='NOC')
    plt.title('Nombre d\'athlètes par pays en 2020')
    plt.xticks(rotation=90)

    plt.tight_layout()
    plt.show()
else:
    print("Les colonnes nécessaires pour créer les graphiques n'existent pas dans le DataFrame.")

   NOMBRETOTALATHLETES GENRE  NOMBREATHLETESPARGENRE  NOC  \
0                 6889  Male                       4  AFG   
1                 6889  Male                       6  ALB   
2                 6889  Male                      26  ALG   
3                 6889  Male                       1  AND   
4                 6889  Male                       2  ANG   
5                 6889  Male                       3  ANT   
6                 6889  Male                      32  ARG   
7                 6889  Male                      14  ARM   
8                 6889  Male                       2  ARU   
9                 6889  Male                       3  ASA   

   NOMBREATHLETESPARPAYS  
0                      4  
1                      6  
2                     26  
3                      1  
4                      2  
5                      3  
6                     32  
7                     14  
8                      2  
9                      3  
Les colonnes nécessaires pour c

##### Répartition par NOC

In [75]:
# Récupérer les données, les transformer si nécessaire, puis afficher sous forme de tableau et/ou d'un graphique
# code fait par Joris 

data = requete_vers_dataframe(conn,f"""
                            SELECT 
                                (SELECT COUNT(DISTINCT A.IdAthlete) AS "Nombre"
                                FROM {SCHEMA}.ATHLETE A
                                INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI ON A.IdAthlete = PI.IdAthlete
                                INNER JOIN {SCHEMA}.EVENEMENT E ON PI.IdEvent = E.IdEvenement
                                INNER JOIN {SCHEMA}.HOTE H ON E.IdHote = H.IdHote
                                WHERE H.AnneeHote = 2020) AS NombreTotalAthletes,
                                A.Genre, 
                                COUNT(DISTINCT A.IdAthlete) AS NombreAthletesParGenre,
                                PI.NOC,
                                COUNT(DISTINCT PI.IdAthlete) AS NombreAthletesParPays
                            FROM {SCHEMA}.ATHLETE A
                            INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI ON A.IdAthlete = PI.IdAthlete
                            INNER JOIN {SCHEMA}.EVENEMENT E ON PI.IdEvent = E.IdEvenement
                            INNER JOIN {SCHEMA}.HOTE H ON E.IdHote = H.IdHote
                            WHERE H.AnneeHote = 2020
                            GROUP BY A.Genre, PI.NOC
                            """)
print(data)

# Vérifier que la colonne 'Genre' existe avant de continuer
if 'Genre' in data.columns:
    plt.figure(figsize=(14, 7))

    # Graphique du nombre d'athlètes par genre
    plt.subplot(1, 2, 1)
    sns.countplot(data=data, x='Genre')
    plt.title('Nombre d\'athlètes par genre en 2020')

    # Graphique du nombre d'athlètes par pays
    plt.subplot(1, 2, 2)
    sns.countplot(data=data, x='NOC')
    plt.title('Nombre d\'athlètes par pays en 2020')
    plt.xticks(rotation=90)

    plt.tight_layout()
    plt.show()
else:
    print("La colonne 'Genre' n'existe pas dans le DataFrame.")

     NOMBRETOTALATHLETES   GENRE  NOMBREATHLETESPARGENRE  NOC  \
0                   6889    Male                       4  AFG   
1                   6889    Male                       6  ALB   
2                   6889    Male                      26  ALG   
3                   6889    Male                       1  AND   
4                   6889    Male                       2  ANG   
..                   ...     ...                     ...  ...   
403                 6889  Female                       8  VIE   
404                 6889  Female                       2  VIN   
405                 6889  Female                       2  YEM   
406                 6889  Female                       2  ZAM   
407                 6889  Female                       1  ZIM   

     NOMBREATHLETESPARPAYS  
0                        4  
1                        6  
2                       26  
3                        1  
4                        2  
..                     ...  
403             

#### Médailles par équipes (épreuves par équipe)

Tableau du TOP 10 par équipe

In [31]:
# Récupérer les données, les transformer si nécessaire, puis afficher sous forme de tableau et/ou d'un graphique
# code fait par Meryem 

data_top10_equipes = requete_vers_dataframe(conn, f'''SELECT EQUIPE.NomEquipe AS "Nom de l'Équipe", 
                                            COUNT(PARTICIPATION_EQUIPE.Medaille) AS "Nombre de Médailles"
                                            FROM {SCHEMA}.EQUIPE
                                            INNER JOIN {SCHEMA}.PARTICIPATION_EQUIPE ON EQUIPE.IdEquipe = PARTICIPATION_EQUIPE.IdEquipe
                                            INNER JOIN {SCHEMA}.EVENEMENT ON PARTICIPATION_EQUIPE.IdEvenement = EVENEMENT.IdEvenement
                                            WHERE EVENEMENT.IdHote = 14
                                            GROUP BY EQUIPE.NomEquipe
                                            ORDER BY "Nombre de Médailles" DESC''')

# Afficher les résultats sous forme de tableau
data_top10_equipes.iloc[0:10] 


,Nom de l'Équipe,Nombre de Médailles
0,Soviet Union,16
1,United States,12
2,Hungary,9
3,Australia,8
4,France,6
5,Unified Team of Germany,6
6,Italy,5
7,Great Britain,4
8,Finland,3
9,Poland,2


Répartition par NOC

In [32]:
# Récupérer les données, les transformer si nécessaire, puis afficher sous forme de tableau
# code fait par Meryem 

data_medailles_noc_equipe = requete_vers_dataframe(conn, f'''SELECT NOC.NOMNOC AS "Nom du NOC", COUNT(*) AS "Nombre de Médailles" 
                                                   FROM NOC 
                                                   INNER JOIN EQUIPE ON NOC.CodeNOC = EQUIPE.NOC 
                                                   INNER JOIN PARTICIPATION_EQUIPE ON EQUIPE.IdEquipe = PARTICIPATION_EQUIPE.IdEquipe 
                                                   INNER JOIN EVENEMENT ON PARTICIPATION_EQUIPE.IdEvenement = EVENEMENT.IdEvenement 
                                                   WHERE IdHote=14 
                                                   GROUP BY NOC.NOMNOC 
                                                   ORDER BY "Nombre de Médailles" DESC''')


data_medailles_noc_equipe


,Nom du NOC,Nombre de Médailles
0,Australia,34
1,United States,34
2,Soviet Union,31
3,Germany,23
4,Great Britain,23
5,France,21
6,Italy,20
7,Hungary,15
8,Canada,14
9,Sweden,12


## Partie modification des données

On décide d'ajouter à l'édition des JO étudiée une nouvelle épreuve : le tir à l'arc à poulies. C'est une épreuve de tir à l'arc (*archery*, code `ARC`), qui se fait soit en arc à poulies homme (individuel ou équipe, mais on ne traitera que le cas individuel), soit arc à poulies femmes (individuel ou équipe). Les noms à utiliser pour les événements seront :

- *Compound individual, Men*
- *Compound individual, Women*

Pour simuler les résultats, vous utiliserez les résultats de l'épreuve de [tir à l'arc classique des JO de 2020 en individuel homme](https://www.olympedia.org/results/18000492) en faisant comme s'il s'agissait d'arc à poulies : les athlètes existent déjà dans la base, vous devez maintenant faire en sorte d'ajouter à la base les différents résultats.

Avant de réaliser les modifications, affichez le tableau des médailles individuelles pour l'Italie

In [15]:
# On donne ici l'exemple de comment ajouter un événement. On a stocké dans id_hote l'identifiant des JO sur lesquels on travaille
with Connexion(login=conn['login'], password=conn['password']) as connexion:  # Démarre une nouvelle connexion
    # On démarre une nouvelle transaction
    connexion.begin()
    # On ouvre un curseur, qui est une sorte de pointeur vers une table de résultats
    curseur = connexion.cursor()
    # On passe la requête paramétrée et les paramètres (s'il y en a) au curseur
    curseur.execute(f"SELECT MAX(IDEVENEMENT) FROM {SCHEMA}.EVENEMENT")
    # S'il n'y a pas d'erreur, on peut demander à récupérer les résultats : 
    # - curseur.fetchone() récupère la ligne sur laquelle pointe le curseur, renvoie le résultat sous forme de tuple, et avance à la ligne suivante (si elle existe)
    # - curseur.fetchall() boucle sur toutes les lignes restantes et renvoie une liste de tuples, ou une liste vide s'il n'y a plus de résultats
    # - curseur.fetchmany(nb) renvoie les résultats des nb lignes suivantes sous forme de liste de tuples
    idev = curseur.fetchone()[0]  # Renvoie un tuple, on veut le premier élément du tuple
    idev += 1
    curseur.execute(f"INSERT INTO {SCHEMA}.EVENEMENT VALUES(:idev, :nomev, :statutev, :codedisc, :idhote)", {'idev': idev, 'nomev': 'Compound Individual, Men', 'statutev': 'Olympic', 'codedisc': 'ARC', 'idhote': id_hote})
    connexion.rollback()  # On annule, ce qui est au dessus est pour l'exemple, vous devrez utiliser commit() pour valider la transaction.

In [37]:
# Afficher le tableau des médailles individuelles de l'Italie
# code fait par Meryem 
#pas sûre à vérifier

data_medailles_individuelles_italie = requete_vers_dataframe(conn, f'''SELECT ATHLETE.NomAthlete AS "Nom de l'Athlète",
                                                             ATHLETE.PrenomAthlete AS "Prenom de l'Athlète",
                                                             EVENEMENT.NomEvenement AS "Épreuve", 
                                                             PARTICIPATION_INDIVIDUELLE.Medaille AS "Médaille"
                                                             FROM {SCHEMA}.ATHLETE
                                                             INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE ON ATHLETE.IdAthlete = PARTICIPATION_INDIVIDUELLE.IdAthlete
                                                             INNER JOIN {SCHEMA}.EVENEMENT ON PARTICIPATION_INDIVIDUELLE.IdEvent = EVENEMENT.IdEvenement
                                                             INNER JOIN {SCHEMA}.NOC ON PARTICIPATION_INDIVIDUELLE.NOC = NOC.CodeNOC
                                                             INNER JOIN {SCHEMA}.HOTE ON HOTE.IdHote = EVENEMENT.IdHote                                                            
                                                             WHERE NOC.NomNOC = 'Italy'
                                                             AND PARTICIPATION_INDIVIDUELLE.Medaille IS NOT NULL
                                                             AND LibelleHote='2020 Summer OLympics'
                                                             ORDER BY Medaille DESC, NomAthlete ''')

data_medailles_individuelles_italie

,Nom de l'Athlète,Prenom de l'Athlète,Épreuve,Médaille


### Résultats des médailles

Pour terminer, saisir les résultats des médailles

In [ ]:
# Faire ici les insertions de données pour les résultats dans la table PARTICIPATION_INDIVIDUELLE
# Code fait par Meryem mais ne fonctionne pas comme je le souhaite

with Connexion(login=conn['login'], password=conn['password']) as connexion:

    connexion.begin()

    curseur = connexion.cursor()
    
    curseur.execute("ALTER TRIGGER trigger1 DISABLE")

    curseur.execute(f'''SELECT ATHLETE.IdAthlete, 
                    ATHLETE.NomAthlete, 
                    ATHLETE.PrenomAthlete,                                                                
                    PARTICIPATION_INDIVIDUELLE.Resultat, 
                    TO_NUMBER(REPLACE(PARTICIPATION_INDIVIDUELLE.Resultat, '=', '')) AS resultatNUM,
                    PARTICIPATION_INDIVIDUELLE.NOC
                    FROM {SCHEMA}.ATHLETE
                    INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE ON ATHLETE.IdAthlete = PARTICIPATION_INDIVIDUELLE.IdAthlete
                    INNER JOIN {SCHEMA}.EVENEMENT ON PARTICIPATION_INDIVIDUELLE.IdEvent = EVENEMENT.IdEvenement
                    WHERE IdEvenement=90016773
                    ORDER BY Resultat ASC, ATHLETE.NomAthlete, ATHLETE.PrenomAthlete''')  

    
    tab = curseur.fetchall()
    
    for i in range(len(tab)):
        ligne = tab[i]
        idat = ligne[0]
        resultat = ligne[3]
        resultatnum = ligne[4]
        NOC = ligne[5]       
        medaille =None
        if resultatnum == 3:  
            medaille ='Bronze'
        elif resultatnum == 2:
            medaille ='Silver'
        elif resultatnum == 1:
            medaille ='Gold'

    connexion.commit()

    curseur.execute(f"INSERT INTO {SCHEMA}.PARTICIPATION_INDIVIDUELLE VALUES (:IdAthlete, :IdEvent, :Resultat, :Medaille, :NOC)",
                {'IdAthlete': 111536, 'idEvent': '90016773', 'Resultat': '2', 'Medaille': 'Medaille', 'NOC': 'ITA'})
    
    curseur.execute(f"INSERT INTO {SCHEMA}.PARTICIPATION_INDIVIDUELLE VALUES (:IdAthlete, :IdEvent, :Resultat, :Medaille, :NOC)",
                {'IdAthlete': 26167, 'idEvent': '90016773', 'Resultat': '3', 'Medaille': 'Medaille', 'NOC': 'JAP'})
    
    curseur.execute(f"INSERT INTO {SCHEMA}.PARTICIPATION_INDIVIDUELLE VALUES (:IdAthlete, :IdEvent, :Resultat, :Medaille, :NOC)", 
                {'IdAthlete': 136120, 'idEvent': '90016773', 'Resultat': '1', 'Medaille': 'Medaille', 'NOC': 'TUR'})

    
    curseur.execute("ALTER TRIGGER trigger1 ENABLE")
    

    
    data_epreuve = requete_vers_dataframe(conn, f'''SELECT ATHLETE.IdAthlete, 
                                           ATHLETE.NomAthlete, 
                                           ATHLETE.PrenomAthlete,                                                                
                                           PARTICIPATION_INDIVIDUELLE.Resultat, 
                                           TO_NUMBER(REPLACE(PARTICIPATION_INDIVIDUELLE.Resultat, '=', '')) AS resultatNUM,
                                           PARTICIPATION_INDIVIDUELLE.Medaille,                                           
                                           PARTICIPATION_INDIVIDUELLE.NOC
                                           FROM {SCHEMA}.ATHLETE
                                           INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE ON ATHLETE.IdAthlete = PARTICIPATION_INDIVIDUELLE.IdAthlete
                                           INNER JOIN {SCHEMA}.EVENEMENT ON PARTICIPATION_INDIVIDUELLE.IdEvent = EVENEMENT.IdEvenement
                                           WHERE IdEvenement=90016773
                                           ORDER BY Resultat ASC, ATHLETE.NomAthlete, ATHLETE.PrenomAthlete''')

data_epreuve    


In [50]:
# Faire ici les insertions de données pour les résultats dans la table PARTICIPATION_INDIVIDUELLE
# Code fait par Meryem
# Comme le code précédent ne fonctionne pas, j'ai essayé une autre méthode d'insertion (celle qu'a utilisé Enzo dans le 2e carnet),
#mais cela n'a pas inséré dans PARTCICIPATION_INDIVIDUELLE

data = {
    "IdAthlete": [
        136120, 111536, 26167
    ],
    "idEvent": [
        90016773, 90016773, 90016773
    ],
    "Resultat": [
        1, 2, 3
    ],
    "Medaille": [
        "Gold", "Bronze", "Silver" 
    ],
    "NOC": [
        "TUR", "ITA", "JAP" 
    ]
}

# Création du DataFrame
df = pd.DataFrame(data)

# organisation des colonnes
df = df[["IdAthlete", "idEvent", "Resultat", "Medaille", "NOC"]]

# Enfin on affiche le DataFrame
df

,IdAthlete,idEvent,Resultat,Medaille,NOC
0,136120,90016773,1,Gold,TUR
1,111536,90016773,2,Bronze,ITA
2,26167,90016773,3,Silver,JAP


## Vérification

Afficher le tableau des médailles de l'Italie après les ajouts de la nouvelle épreuve.

In [51]:
# Affichage du tableau (cf. début de la partie modification)
# Code fait par Meryem

data_medailles_italie_apres_ajouts = requete_vers_dataframe(conn, f'''SELECT ATHLETE.NomAthlete AS "Nom de l'Athlète",
                                                             ATHLETE.PrenomAthlete AS "Prenom de l'Athlète",
                                                             EVENEMENT.NomEvenement AS "Épreuve", 
                                                             PARTICIPATION_INDIVIDUELLE.Medaille AS "Médaille"
                                                             FROM {SCHEMA}.ATHLETE
                                                             INNER JOIN {SCHEMA}.PARTICIPATION_INDIVIDUELLE ON ATHLETE.IdAthlete = PARTICIPATION_INDIVIDUELLE.IdAthlete
                                                             INNER JOIN {SCHEMA}.EVENEMENT ON PARTICIPATION_INDIVIDUELLE.IdEvent = EVENEMENT.IdEvenement
                                                             INNER JOIN {SCHEMA}.NOC ON PARTICIPATION_INDIVIDUELLE.NOC = NOC.CodeNOC
                                                             INNER JOIN {SCHEMA}.HOTE ON HOTE.IdHote = EVENEMENT.IdHote                                                            
                                                             WHERE NOC.NomNOC = 'Italy'
                                                             AND PARTICIPATION_INDIVIDUELLE.Medaille IS NOT NULL
                                                             AND LibelleHote='2020 Summer OLympics'
                                                             ORDER BY Medaille DESC, NomAthlete ''')
                                                                
data_medailles_italie_apres_ajouts

,Nom de l'Athlète,Prenom de l'Athlète,Épreuve,Médaille


### Nettoyage

Supprimez maintenant tous les ajouts précédents (médailles, résultats détaillés, nouvelles épreuves) de votre base.

In [ ]:
# Nettoyage (avec des requêtes DELETE)
# Code fait par Meryem

nettoyage = requete_vers_dataframe(conn, f'''DELETE FROM {SCHEMA}.PARTICIPATION_INDIVIDUELLE
                                    WHERE noc = 'ITA'
                                    AND (IdEvent IN (SELECT IdEvenement FROM {SCHEMA}.EVENEMENT WHERE NomEvenement LIKE 'NomDeLaNouvelleEpreuve%')
                                    OR IdAthlete IN (SELECT IdAthlete FROM {SCHEMA}.ATHLETE WHERE NomAthlete LIKE 'NomDuNouvelAthlete%')
                                    OR Medaille IN ('Gold', 'Silver', 'Bronze'))''')

print("Nettoyage des ajouts précédents effectué")
